* 예제 핵심 : DICOM directory 로 부터 데이터를 추출한다. (DicomDir 클래스 사용)
* <'pydicom.filereader'> read_dicomdir : DicomDir 타입을 반환함 
* <'os.path'> 
    * dirname : 해당 filepath(str 타입)가 포함된 상위 디렉토리명을 str 타입으로 반환 
    * join : 해당 OS 형식에 맞도록 입력 받은 경로를 연결. (입력 중간에 절대경로가 나오면 이전에 취합된 경로는 제거하고 다시 연결)
* <'DicomDir' class> patient_records : <'list('pydicom.dataset.Dataset')'> 타입을 반환 
* <'pydicom.dataset.Dataset' class> children : list타입을 반환 
* hasattr(A, B) : A안에 B 프로퍼티가 존재하는지 확인 
* ('A', 'B')[조건문] : 조건문 결과가 Flase이면 'A', True이면 'B'가 반환. 

In [37]:
# authors : Guillaume Lemaitre <g.lemaitre58@gmail.com>
# license : MIT

from os.path import dirname, join
from pprint import pprint

import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir

# fetch the path to the test data
filepath = get_testdata_files('DICOMDIR')[0]
print('Path to the DICOM directory: {}'.format(filepath))
# load the data
dicom_dir = read_dicomdir(filepath)
base_dir = dirname(filepath)

# go through the patient record and print information
for patient_record in dicom_dir.patient_records:
    if (hasattr(patient_record, 'PatientID') and
            hasattr(patient_record, 'PatientName')):
        print("Patient: {}: {}".format(patient_record.PatientID,
                                       patient_record.PatientName))
    studies = patient_record.children
    # got through each serie
    for study in studies:
        print(" " * 4 + "Study {}: {}: {}".format(study.StudyID,
                                                  study.StudyDate,
                                                  study.StudyDescription))
        all_series = study.children
        # go through each serie
        for series in all_series:
            image_count = len(series.children)
            plural = ('', 's')[image_count > 1]

            # Write basic series info and image count

            # Put N/A in if no Series Description
            if 'SeriesDescription' not in series:
                series.SeriesDescription = "N/A"
            print(" " * 8 + "Series {}: {}: {} ({} image{})".format(
                series.SeriesNumber, series.Modality, series.SeriesDescription,
                image_count, plural))

            # Open and read something from each image, for demonstration
            # purposes. For simple quick overview of DICOMDIR, leave the
            # following out
            print(" " * 12 + "Reading images...")
            image_records = series.children
            image_filenames = [join(base_dir, *image_rec.ReferencedFileID)
                               for image_rec in image_records]
    

            datasets = [pydicom.dcmread(image_filename)
                        for image_filename in image_filenames]

            patient_names = set(ds.PatientName for ds in datasets)
            patient_IDs = set(ds.PatientID for ds in datasets)

            # List the image filenames
            print("\n" + " " * 12 + "Image filenames:")
            print(" " * 12, end=' ')
            pprint(image_filenames, indent=12)

            # Expect all images to have same patient name, id
            # Show the set of all names, IDs found (should each have one)
            print(" " * 12 + "Patient Names in images..: {}".format(
                patient_names))
            print(" " * 12 + "Patient IDs in images..: {}".format(
                patient_IDs))

Path to the DICOM directory: /Users/vuno-bmkim/Documents/project/pydicom/.env/lib/python3.7/site-packages/pydicom/data/test_files/dicomdirtests/DICOMDIR
Patient: 77654033: Doe^Archibald
    Study 2: 20010101: XR C Spine Comp Min 4 Views
        Series 1: CR: N/A (1 image)
            Reading images...

            Image filenames:
             [           '/Users/vuno-bmkim/Documents/project/pydicom/.env/lib/python3.7/site-packages/pydicom/data/test_files/dicomdirtests/77654033/CR1/6154']
            Patient Names in images..: {'Doe^Archibald'}
            Patient IDs in images..: {'77654033'}
        Series 2: CR: N/A (1 image)
            Reading images...

            Image filenames:
             [           '/Users/vuno-bmkim/Documents/project/pydicom/.env/lib/python3.7/site-packages/pydicom/data/test_files/dicomdirtests/77654033/CR2/6247']
            Patient Names in images..: {'Doe^Archibald'}
            Patient IDs in images..: {'77654033'}
        Series 3: CR: N/A (1 image)

* <'pydicom.dataset.Dataset' class> 의 경우 object를 상속받은 클래스이지만,  \_\_getitem\_\_() 함수 구현을 통해 unpacking( * )을 할 수 있다. (참고 : https://stackoverflow.com/questions/37837520/implement-packing-unpacking-in-an-object)

In [30]:
class Item(object):

    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender
        
    def __getitem__(self, index):
        return (self.name, self.age, self.gender)[index]


item = Item("Henry", 90, "male")
print(item)
print(*item)

Henry 90 male
